In [1]:
from itertools import product, combinations
from sage.combinat.ribbon_tableau import *
from sage.combinat.q_analogues import *

Tableaux.options.convention="french"
Sym = SymmetricFunctions(FractionField(QQ['t']))
s = Sym.s()
m = Sym.m()
h = Sym.h()
K.<u> = LaurentPolynomialRing(ZZ)

Univariate Laurent Polynomial Ring in u over Integer Ring

In [2]:
def contragredient(lmbda):
    x = -w0.action(AS(lmbda))
    return WC(x).weight_multiplicities()

def pad(p, n):
    return p + [0 for i in xrange(n-len(p))]

def is_straightened(ket):
    ''' Return True if ``ket`` is strictly dominant weight, else False. '''
    for alpha_check in SCR:
        if ket.scalar(alpha_check.to_ambient()) <= 0:
            return False
    return True

def is_all_straightened(kets):
    ''' Return True if list of weights ``kets`` consists of all strictly 
        dominant weights, else False.
    '''
    for ket in kets:
        if not is_straightened(ket):
            return False
    return True

def Q(nu, lmbda, k):
    mons = contragredient(lmbda)
    kets = {}
    for exp in mons:
        new_ket = AS(nu) - k*exp
        kets[new_ket] = mons[exp]
    while not is_all_straightened(kets): # some ket in kets is not straightened
        # get ket that needs straightening
        for ket in kets:
            if not is_straightened(ket):
                ket_s = ket # ket_s is ket to straighten
                break
        # find i^th simple to straighten at, and get p, r
        for i in IO:
            if ket_s.scalar(SCR[i].to_ambient()) <= 0:
                (p, r) = divmod(-int(ket_s.scalar(SCR[i].to_ambient())), int(k))
                s_index = i
                break
        # straighten and add new kets to dict, remove old ket
        if r==0 and p==0:
            kets.pop(ket_s)
        elif r==0 and p>0:
            ket_s_trans = ket_s.simple_reflection(s_index) # apply transposition to ket_s
            kets[ket_s_trans] = kets.get(ket_s_trans, 0) - kets[ket_s]
            kets.pop(ket_s)
        elif r!=0 and p==0:
            ket_s_trans = ket_s.simple_reflection(s_index) # apply transposition to ket_s
            kets[ket_s_trans] = kets.get(ket_s_trans, 0) + (u^-1)*kets[ket_s]
            kets.pop(ket_s)
        elif r!=0 and p>0:
            ket_s_trans1 = ket_s.simple_reflection(s_index) # apply transposition to ket_s
            ket_s_trans2 = ket_s + r*SR[s_index].to_ambient()
            ket_s_trans3 = ket_s.simple_reflection(s_index) - r*SR[s_index].to_ambient()
            kets[ket_s_trans1] = kets.get(ket_s_trans1, 0) + (u^-1)*kets[ket_s]
            kets[ket_s_trans2] = kets.get(ket_s_trans2, 0) + (u^-1)*kets[ket_s]
            kets[ket_s_trans3] = kets.get(ket_s_trans3, 0) - kets[ket_s]
            kets.pop(ket_s)
    return kets

def Q_mu(mu, nu, lmbda, k):
    kets = Q(nu, lmbda, k)
    return kets.get(AS(mu), 0)

# doesn't currently do Type A, since relies on Q which is Type independent
def LLT_A(mu, nu, n, k):
    llt = 0
    size = int( (mu.height() - nu.height()) / k )
    for lmbda in Partitions(size, max_length=n):
        lmbda = pad(lmbda, n)
        llt += Q_mu(mu, nu, lmbda, k)*s[lmbda]
    # STILL NEED TO ACCOUNT FOR POWER OF U IN FRONT
    return llt

# Takes in partition or ambient space element,
# outputs element of weight lattice (not just weight space) if co=False
# and coweight lattice if co=True; False by default.
# CAUTION: in type A, this will make last component 0 (map to sl_n weight space)
def weight_lattice(lmbda, co=False):
    if co:
        return CWL.from_vector(AS(lmbda).to_weight_space().to_vector())
    else:
        return WL.from_vector(AS(lmbda).to_weight_space().to_vector())

# helper function for LLT
# input: parab, beta, gamma, n
# beta, gamma are strictly dominant weights
# output: mu, nu, power to calculate Q coeffs
# mu, nu strictly dominant weights
# in the process, calculates eta, k
def from_parab_to_dominant(parab, beta, gamma):
    # convert beta, gamma to translation in extended Weyl
    if Type == 'A':
        tau_beta = W_PW0(AS(beta))
        tau_gamma = W_PW0(AS(gamma))
    else:
        tau_beta = W_d_PvW0(weight_lattice(beta))
        tau_gamma = W_d_PvW0(weight_lattice(gamma))
    # find minimal coset reps for beta, gamma
    v = tau_beta.coset_representative(IO, side='right').coset_representative(index_set=parab, side='left')
    w = tau_gamma.coset_representative(IO, side='right').coset_representative(index_set=parab, side='left')
    # calculate eta, k
    eta = AS(0)
    for fw in IO:
        if fw not in parab:
            eta -= AS.fundamental_weights()[fw]
    k = int(1 - weight_lattice(eta).scalar(RSp.highest_root().max_coroot_le())) # doesn't yet check condition(b) on pg 13
    # calculate mu, nu
    x = W_d_PvW0.from_classical_weyl(w0)*v.inverse()
    y = W_d_PvW0.from_classical_weyl(w0)*w.inverse()
    (xt, xs) = x.to_dual_translation_left(), x.to_dual_classical_weyl()
    (yt, ys) = y.to_dual_translation_left(), y.to_dual_classical_weyl()
    if Type != 'A':
        eta = weight_lattice(eta)
    mu = xs.action(eta) - k*xt
    nu = ys.action(eta) - k*yt
    # compute Q, add to llt
    power = w.length() - v.length()
    if Type == 'A': # to compare to combinatorial LLTs
        power += x.length() - y.length()
    else:
        t = weight_lattice(beta) - weight_lattice(gamma)
        power += W_d_PvW0(W_d_FW(t).to_fundamental_group()).to_dual_classical_weyl().length()
    return (mu, nu, power, k)

# inputs parabolic, beta, gamma, n, size=cutoff for LLT
# for Type A, will only output polynomial part
# uncommment below to index over dominant weights and not partitions
def LLT(parab, beta, gamma, n, size):
    (mu, nu, power, k) = from_parab_to_dominant(parab, beta, gamma)
    llt = {}
    
    # Index over dominant weights up to some threshold
#     llt2 = {}
#     Lambda = WL.fundamental_weights()
#     max_ht = int(2*size / (n*(n+1))) + 1 + 1
#     for dw in product(range(max_ht), repeat=len(IO)):
#         dom_wt = sum([dw[j]*Lambda[j+1] for j in xrange(len(dw))])
#         coeff = Q_mu(mu, nu, dom_wt, n, k)
#         if coeff != 0:
#             llt2[WC(dom_wt)] = coeff * (u^power)

    # Index over partitions
    for lmbda in Partitions(size, max_length=n):
        lmbda = pad(lmbda, n)
        coeff = Q_mu(mu, nu, lmbda, k)
        if coeff != 0:
            llt[WC(lmbda)] = coeff * (u^power)
    return llt

# returns LLT (by indexing over partitions not dominant weights) for any Type
# Returns only polynomial part for type A
# mu, nu are strictly dominant weights
def LLT_mu_nu(mu, nu, power, n, k, size):
    llt = {}
    for lmbda in Partitions(size, max_length=n):
        lmbda = pad(lmbda, n)
        coeff = Q_mu(mu, nu, lmbda, k)
        if coeff != 0:
            llt[WC(lmbda)] = coeff * (u^power)
    return llt

# returns LLT for type A without truncating to polynomial part
def LLT_GLn(mu, nu, power, n, k, size):
    llt = {}
    for lmbda in Partitions(size, max_length=n):
        lmbda = pad(lmbda, n)
        # get coeff for lmbda
        coeff = Q_mu(mu, nu, lmbda, k)
        if coeff != 0:
            llt[WC(lmbda)] = coeff * (u^power)
        # get coeff for dominant wts from lmbda that aren't partitions
        for i in range(lmbda[-1]+1, lmbda[0]+1):
            lmbda_det = AS(lmbda) - i*DET
            coeff = Q_mu(mu, nu, lmbda_det, k)
            if coeff != 0:
                llt[WC(lmbda_det)] = coeff * (u^power)
    return llt

In [11]:
###########################
# GLOBAL VARIABLES
###########################

n = 2
Type = 'C'
# Type A indexes differently
if Type == 'A':
    RootType = [Type, n-1]
    gl = True # for extended affine, want GLn
else:
    RootType = [Type, n]
    gl = False
#
RS = RootSystem(RootType)
RS_d = RootSystem(CartanType(RootType).dual())
RSp = RS.root_space()
AS = RS.ambient_space()
RL = RS.root_lattice()
WL = RS.weight_lattice()
CWL = RS.coweight_lattice()
WL_d = RS_d.weight_lattice()
SR = RL.simple_roots()
SCR = RL.simple_coroots()
W = WeylGroup(RootType)
w0 = W.long_element()
WC = WeylCharacterRing(RootType)
wc = WeightRing(WC)
E = ExtendedAffineWeylGroup(RootType, general_linear=gl)
E_d = ExtendedAffineWeylGroup(CartanType(RootType).dual(), general_linear=gl)
W_PW0 = E.PW0()
W_FW = E.FW()
W_d_PvW0 = E_d.PvW0()
W_d_FW = E_d.FW()
IO = CartanType(RootType).index_set()
rho = WL.rho()
rho_n = -w0.action(AS(rho))
if Type == 'A':
    DET = AS.det() # determinant for switching between SLn and GLn
else:
    DET = 0


###########################
# SET INPUTS
###########################

print "------------------------------------------------------------"
print "Testing General LLT"
print "------------------------------------------------------------"

'''
parab = set of simple roots
beta, gamma = strictly dominant weights for given parab
'''
parab = [1]
beta = [2,1]
gamma = [1,0]

(mu, nu, power, k) = from_parab_to_dominant(parab, beta, gamma)
print "parab: {0}, beta: {1}, gamma: {2}".format(parab, beta, gamma)
print "mu: {0}, nu: {1}, power: {2}, n: {3}, k: {4}".format(mu, nu, power, n, k)

'''
manually set mu, nu, k if need be
'''
# mu = 
# nu = 
# k =

for size in range(0,9,1):
    print "size: ", size
    llt = {}
    for lmbda in Partitions(size, max_length=n):
        lmbda = pad(lmbda, n)
        coeff = Q_mu(mu, nu, lmbda, k)
        if coeff != 0:
            llt[WC(lmbda)] = coeff * (u^power)
            print "lmbda: ", lmbda
            print "coeff: ", coeff * (u^power)
    print "LLT: ", llt

    print "running LLT: ", LLT(parab, beta, gamma, n, size)
    #print "LLT_GLN: ", LLT_GLn(mu, nu, power, n, k, size)
    print "-------------------------------"
    

    
##########################################################################
print "------------------------------------------------------------"
print "Finished Testing"
print "------------------------------------------------------------"
##########################################################################

------------------------------------------------------------
Testing General LLT
------------------------------------------------------------
parab: [1], beta: [2, 1], gamma: [1, 0]
mu: 3*Lambda[1] + 2*Lambda[2], nu: Lambda[1] + Lambda[2], power: -3, n: 2, k: 3
size:  0
LLT:  {}
running LLT:  {}
-------------------------------
size:  1
LLT:  {}
running LLT:  {}
-------------------------------
size:  2
lmbda:  [1, 1]
coeff:  u^-4
LLT:  {C2(1,1): u^-4}
running LLT:  {C2(1,1): u^-4}
-------------------------------
size:  3
LLT:  {}
running LLT:  {}
-------------------------------
size:  4
lmbda:  [3, 1]
coeff:  u^-6
LLT:  {C2(3,1): u^-6}
running LLT:  {C2(3,1): u^-6}
-------------------------------
size:  5
LLT:  {}
running LLT:  {}
-------------------------------
size:  6
lmbda:  [5, 1]
coeff:  u^-8
lmbda:  [3, 3]
coeff:  u^-8
LLT:  {C2(5,1): u^-8, C2(3,3): u^-8}
running LLT:  {C2(5,1): u^-8, C2(3,3): u^-8}
-------------------------------
size:  7
LLT:  {}
running LLT:  {}
--------------